In [9]:
from typing import Type
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_community.tools.ddg_search.tool import DuckDuckGoSearchResults
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from pydantic import BaseModel, Field
from langchain_community.utilities.duckduckgo_search import DuckDuckGoSearchAPIWrapper

import wikipedia
from langchain.tools import BaseTool


class DuckDuckGoSearchToolArgsSchema(BaseModel):
    query: str = Field(
        title="Query",
        description="The search query to search for",
    )


class DuckDuckGoSearchTool(BaseTool):
    name = "DuckDuckGoSearchTool"
    description = "Searches DuckDuckGo for the top search results"
    args_schema: Type[DuckDuckGoSearchToolArgsSchema] = DuckDuckGoSearchToolArgsSchema

    def _run(self, query: str):
        duckduckgo = DuckDuckGoSearchAPIWrapper()
        return duckduckgo.run(query)


wiki_wrapper = WikipediaAPIWrapper(
    wiki_client=wikipedia, top_k_results=1, doc_content_chars_max=1000
)

llm = ChatOpenAI(temperature=0.1)
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()
tools = [DuckDuckGoSearchTool(), WikipediaQueryRun(api_wrapper=wiki_wrapper)]

agent = create_openai_tools_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor.invoke({"input": "Research about the XZ backdoor"})

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `XZ backdoor`


Page: XZ Utils backdoor
Summary: On 29 March 2024, software developer Andres Freund reported that he had found a maliciously introduced backdoor in the Linux utility xz within the liblzma library in versions 5.6.0 and 5.6.1 released in February 2024.While xz is commonly present in most Linux distributions, the backdoor only targeted Debian- and RPM-based systems running on the x86-64 architecture. At the time of discovery the backdoored version had not yet been widely deployed to production systems, but was pr

{'input': 'Research about the XZ backdoor',
 'output': 'The XZ backdoor refers to a maliciously introduced backdoor found in the Linux utility xz within the liblzma library. The backdoor was reported on 29 March 2024 by software developer Andres Freund. It was present in versions 5.6.0 and 5.6.1 of xz released in February 2024. The backdoor specifically targeted Debian- and RPM-based systems running on the x86-64 architecture.\n\nAlthough xz is commonly found in most Linux distributions, the backdoor only affected specific systems. The backdoor allowed an attacker with a specific Ed448 private key to execute remote code on the affected Linux systems. The issue was assigned a CVSS score of 10.0, the highest possible score. At the time of discovery, the backdoored version had not been widely deployed to production systems but was present in development versions of major distributions.'}